In [1]:
#data loader
#if training data set is very huge , batch size is necessary
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader

class DiabetesDataset(Dataset):
    """Diabetes dataset."""
    
    #Initialize your data, download ,etc
    def __init__(self):
        xy=np.loadtxt('./data/diabetes.csv.gz',delimiter=',',dtype=np.float32)
        self.len=xy.shape[0]
        self.x_data=torch.from_numpy(xy[:,0:-1])
        self.y_data=torch.from_numpy(xy[:,[-1]])
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len
    
dataset=DiabetesDataset()

train_loader=DataLoader(dataset=dataset,
                       batch_size=32,
                       shuffle=True,
                       num_workers=2)
#print the dataset and different batchs
for epoch in range(2):
    for i,data in enumerate(train_loader,0):
        #get the input
        inputs,labels=data
        
        #wrap them in the Variable
        inputs,labels=Variable(inputs),Variable(labels)
        
        #run your training process
        print(epoch,i,"inputs",inputs.data,"labels",labels.data)

0 0 inputs 
-0.5294  0.4573  0.3443 -0.6364  0.0000 -0.0313 -0.8659  0.6333
-0.2941  0.9095  0.5082  0.0000  0.0000  0.0581 -0.8292  0.5000
-0.8824  0.1658  0.2787 -0.4141 -0.5745  0.0760 -0.6430 -0.8667
 0.0588 -0.4271  0.3115 -0.2525  0.0000 -0.0224 -0.9846 -0.3333
 0.0588  0.2362  0.1475 -0.1111 -0.7778 -0.0134 -0.7472 -0.3667
-0.0588 -0.0452  0.1803  0.0000  0.0000  0.0969 -0.6524  0.2000
-0.8824 -0.0050 -0.0492 -0.7980  0.0000 -0.2429 -0.5961  0.0000
-0.7647  0.0050  0.1148 -0.4949 -0.8322  0.1475 -0.7899 -0.8333
-0.8824  0.8191  0.0492 -0.3939 -0.5745  0.0164 -0.7865 -0.4333
-0.6471 -0.2161  0.1475  0.0000  0.0000 -0.0313 -0.8360 -0.4000
-0.0588 -0.0050  0.3770  0.0000  0.0000  0.0551 -0.7353 -0.0333
-0.8824  0.0854  0.4426 -0.6162  0.0000 -0.1922 -0.7250 -0.9000
-0.7647  0.1256  0.2295 -0.3535  0.0000  0.0641 -0.9402  0.0000
 0.0000 -0.0653 -0.0164 -0.4949 -0.7825 -0.1446 -0.6123 -0.9667
-0.1765  0.7889  0.3770  0.0000  0.0000  0.1893 -0.7839 -0.3333
-0.7647  0.0854  0.0492  0.0

1 0 inputs 
-0.5294  0.9799  0.1475 -0.2121  0.7589  0.0939  0.9223 -0.6667
-0.1765  0.0653 -0.0164 -0.5152  0.0000 -0.2101 -0.8138 -0.7333
 0.0000 -0.0553  0.0000  0.0000  0.0000  0.0000 -0.8480 -0.8667
-0.1765  0.0352  0.0820 -0.3535  0.0000  0.1654 -0.7728 -0.6667
-0.7647  0.1256  0.2295 -0.3535  0.0000  0.0641 -0.9402  0.0000
-0.7647  0.2060  0.2459 -0.2525 -0.7518  0.1833 -0.8830 -0.7333
 0.7647  0.3668  0.1475 -0.3535 -0.7400  0.1058 -0.9360 -0.2667
-0.4118  0.1759  0.4098 -0.3939 -0.7518  0.1654 -0.8523 -0.3000
-0.1765  0.8794 -0.1803 -0.3333 -0.0733  0.0104 -0.3612 -0.5667
-0.6471  0.0050  0.1148 -0.5354 -0.8085 -0.0581 -0.2562 -0.7667
-0.8824  1.0000  0.2459 -0.1313  0.0000  0.2787  0.1238 -0.9667
-0.7647 -0.0050  0.0000  0.0000  0.0000 -0.3383 -0.9744 -0.9333
 0.1765  0.2965  0.2459 -0.4343 -0.7116  0.0700 -0.8275 -0.4000
-0.7647  0.1256  0.2787  0.0101 -0.6690  0.1744 -0.9172 -0.9000
 0.2941  0.3869  0.2459  0.0000  0.0000 -0.0104 -0.7079 -0.5333
-0.8824  0.2161  0.2787 -0.2

In [3]:
#create the model class
class Model(torch.nn.Module):
    
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model,self).__init__()
        self.l1=torch.nn.Linear(8,6)
        self.l2=torch.nn.Linear(6,4)
        self.l3=torch.nn.Linear(4,1)
        
        self.sigmoid=torch.nn.Sigmoid()
        
    def forward(self,x):
        out1=self.sigmoid(self.l1(x))
        out2=self.sigmoid(self.l2(out1))
        y_pred=self.sigmoid(self.l3(out2))
        
        return y_pred

#our model
model=Model()

#Construct our loss function and an Optimizer. 
criterion=torch.nn.BCELoss(size_average=True)
optimizer=torch.optim.SGD(model.parameters(),lr=0.1)

#training loop
for epoch in range(100):
    for i,data in enumerate(train_loader,0):
        #get the inputs 
        inputs, labels = data
        
        #wrap them in Variable
        inputs, labels = Variable(inputs),Variable(labels)
        
        #forward pass: compute predicted y by passing x to the model
        y_pred = model(inputs)
        
        #compute and print loss
        loss = criterion(y_pred,labels)
        print(epoch, i , loss.data[0])
        
        #Zero gradients ,perform a backward pass ,and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 0.644388735294342
0 1 0.6620460152626038
0 2 0.6445604562759399
0 3 0.6444480419158936
0 4 0.6263421773910522
0 5 0.7167894840240479
0 6 0.60979825258255
0 7 0.6985474824905396
0 8 0.6446370482444763
0 9 0.6267816424369812
0 10 0.7340061664581299
0 11 0.6785809993743896
0 12 0.6121463179588318
0 13 0.6793252229690552
0 14 0.7127096056938171
0 15 0.5647569298744202
0 16 0.6796779036521912
0 17 0.6447461843490601
0 18 0.6793162226676941
0 19 0.5762747526168823
0 20 0.5713917016983032
0 21 0.6245245337486267
0 22 0.5850836038589478
0 23 0.646350085735321
1 0 0.6841415166854858
1 1 0.5843448638916016
1 2 0.643862783908844
1 3 0.7256202697753906
1 4 0.7023370862007141
1 5 0.643680989742279
1 6 0.624885082244873
1 7 0.6434895396232605
1 8 0.6630155444145203
1 9 0.6629119515419006
1 10 0.6437207460403442
1 11 0.6816832423210144
1 12 0.6253694295883179
1 13 0.5872983336448669
1 14 0.6436580419540405
1 15 0.6437625885009766
1 16 0.6239903569221497
1 17 0.6638187170028687
1 18 0.623884320259

14 9 0.6088581085205078
14 10 0.6072760224342346
14 11 0.5868565440177917
14 12 0.624030351638794
14 13 0.6433815956115723
14 14 0.7230919003486633
14 15 0.6436446309089661
14 16 0.6054639220237732
14 17 0.6630640029907227
14 18 0.6239529252052307
14 19 0.6827519536018372
14 20 0.5860682725906372
14 21 0.6434624195098877
14 22 0.6032683253288269
14 23 0.5889303684234619
15 0 0.5802860260009766
15 1 0.6652389764785767
15 2 0.6007225513458252
15 3 0.7093883156776428
15 4 0.6224493384361267
15 5 0.6222435832023621
15 6 0.6435272693634033
15 7 0.5368767380714417
15 8 0.7563790678977966
15 9 0.6434264183044434
15 10 0.7067975997924805
15 11 0.6026082634925842
15 12 0.5600752830505371
15 13 0.6652687191963196
15 14 0.7504743337631226
15 15 0.5830169916152954
15 16 0.6642807126045227
15 17 0.6434158682823181
15 18 0.7052568197250366
15 19 0.7425974607467651
15 20 0.6249480843544006
15 21 0.7196094393730164
15 22 0.589730441570282
15 23 0.5404905080795288
16 0 0.6634662747383118
16 1 0.6633821

28 8 0.6613821983337402
28 9 0.6265650391578674
28 10 0.6614565849304199
28 11 0.678739607334137
28 12 0.6095672249794006
28 13 0.6793975830078125
28 14 0.6088466644287109
28 15 0.6074235439300537
28 16 0.5869659185409546
28 17 0.6428712010383606
28 18 0.6235409379005432
28 19 0.5838600397109985
28 20 0.6232060194015503
28 21 0.6431980729103088
28 22 0.5812644958496094
28 23 0.6463905572891235
29 0 0.7498121857643127
29 1 0.5826738476753235
29 2 0.6221107840538025
29 3 0.6431282162666321
29 4 0.5385729074478149
29 5 0.5773952007293701
29 6 0.734470009803772
29 7 0.6429492235183716
29 8 0.5573144555091858
29 9 0.6884611248970032
29 10 0.6436659693717957
29 11 0.6868749260902405
29 12 0.6221044659614563
29 13 0.7288768887519836
29 14 0.6839904189109802
29 15 0.6033767461776733
29 16 0.6024659276008606
29 17 0.6638218760490417
29 18 0.6431748867034912
29 19 0.6431853771209717
29 20 0.5407885313034058
29 21 0.6863661408424377
29 22 0.7059324383735657
29 23 0.7309308052062988
30 0 0.6041820

42 9 0.581616997718811
42 10 0.5593182444572449
42 11 0.7741473913192749
42 12 0.6836830377578735
42 13 0.662523627281189
42 14 0.6232523322105408
42 15 0.6226590275764465
42 16 0.6629377007484436
42 17 0.7220844030380249
42 18 0.6615533232688904
42 19 0.6427607536315918
42 20 0.5673263669013977
42 21 0.62235426902771
42 22 0.7241888642311096
42 23 0.698788583278656
43 0 0.6426659822463989
43 1 0.6616001129150391
43 2 0.5686474442481995
43 3 0.603529691696167
43 4 0.541850209236145
43 5 0.6001888513565063
43 6 0.6644495725631714
43 7 0.5787853598594666
43 8 0.5997653007507324
43 9 0.5986897349357605
43 10 0.5980812907218933
43 11 0.7132460474967957
43 12 0.6430765986442566
43 13 0.6215115189552307
43 14 0.6656123399734497
43 15 0.6428096294403076
43 16 0.7517348527908325
43 17 0.6219660639762878
43 18 0.7465687394142151
43 19 0.701101541519165
43 20 0.6240459084510803
43 21 0.5848721861839294
43 22 0.8020855784416199
43 23 0.7214961647987366
44 0 0.7292330265045166
44 1 0.5967360734939

56 10 0.6632258892059326
56 11 0.702947735786438
56 12 0.7017227411270142
56 13 0.6037942171096802
56 14 0.6616912484169006
56 15 0.6614401936531067
56 16 0.6797211766242981
56 17 0.623101532459259
56 18 0.5856541991233826
56 19 0.7016310095787048
56 20 0.6233252286911011
56 21 0.6415033936500549
56 22 0.5449823141098022
56 23 0.675135612487793
57 0 0.6225332617759705
57 1 0.6629172563552856
57 2 0.6215512752532959
57 3 0.662647545337677
57 4 0.6018453240394592
57 5 0.5594146847724915
57 6 0.6205826997756958
57 7 0.6867730021476746
57 8 0.8123354315757751
57 9 0.6230716109275818
57 10 0.7003027200698853
57 11 0.6614758372306824
57 12 0.5477957725524902
57 13 0.620733380317688
57 14 0.601504385471344
57 15 0.621116042137146
57 16 0.642087996006012
57 17 0.6005353331565857
57 18 0.5791106224060059
57 19 0.6644477248191833
57 20 0.7294768691062927
57 21 0.7445051670074463
57 22 0.6432271599769592
57 23 0.6445307731628418
58 0 0.5834431648254395
58 1 0.5816052556037903
58 2 0.7050449848175

70 14 0.6402990818023682
70 15 0.6021406054496765
70 16 0.642430305480957
70 17 0.723348081111908
70 18 0.6606590151786804
70 19 0.5843852758407593
70 20 0.6405825018882751
70 21 0.6420110464096069
70 22 0.7199884057044983
70 23 0.6721641421318054
71 0 0.6802325248718262
71 1 0.6404175758361816
71 2 0.677174985408783
71 3 0.6421129107475281
71 4 0.6056028604507446
71 5 0.6595233678817749
71 6 0.622879147529602
71 7 0.6231225728988647
71 8 0.6610146760940552
71 9 0.6609908938407898
71 10 0.6988074779510498
71 11 0.5497549772262573
71 12 0.6213064789772034
71 13 0.6399146318435669
71 14 0.5413108468055725
71 15 0.6616125106811523
71 16 0.5793293118476868
71 17 0.7277718782424927
71 18 0.5801382660865784
71 19 0.726343035697937
71 20 0.5822362899780273
71 21 0.7244172096252441
71 22 0.6623385548591614
71 23 0.6981819272041321
72 0 0.6594764590263367
72 1 0.5846256613731384
72 2 0.6405267715454102
72 3 0.679481029510498
72 4 0.6420621871948242
72 5 0.6227535605430603
72 6 0.660483598709106

85 0 0.6190405488014221
85 1 0.6822267174720764
85 2 0.5997744798660278
85 3 0.8047465682029724
85 4 0.7151126861572266
85 5 0.6223382353782654
85 6 0.6948838233947754
85 7 0.6040622591972351
85 8 0.6594168543815613
85 9 0.6396744847297668
85 10 0.5672545433044434
85 11 0.6583034992218018
85 12 0.6777912378311157
85 13 0.5863221883773804
85 14 0.5803178548812866
85 15 0.5392382740974426
85 16 0.5753012299537659
85 17 0.683766782283783
85 18 0.6187558770179749
85 19 0.6849945783615112
85 20 0.5765860676765442
85 21 0.6852378845214844
85 22 0.6394055485725403
85 23 0.7293842434883118
86 0 0.6806461215019226
86 1 0.6994099020957947
86 2 0.6019660234451294
86 3 0.6594038009643555
86 4 0.6588617563247681
86 5 0.6585322618484497
86 6 0.6002518534660339
86 7 0.6789066791534424
86 8 0.5814319849014282
86 9 0.6612488627433777
86 10 0.6184354424476624
86 11 0.6191192865371704
86 12 0.5810020565986633
86 13 0.7430612444877625
86 14 0.6766266226768494
86 15 0.6603630781173706
86 16 0.5849890708923

99 0 0.6383024454116821
99 1 0.5609945058822632
99 2 0.7328717112541199
99 3 0.6732877492904663
99 4 0.7090215682983398
99 5 0.5733369588851929
99 6 0.6769516468048096
99 7 0.635859489440918
99 8 0.6710776090621948
99 9 0.6391438245773315
99 10 0.7277973294258118
99 11 0.6866155862808228
99 12 0.6715064644813538
99 13 0.6103493571281433
99 14 0.6194353103637695
99 15 0.5859621167182922
99 16 0.5841492414474487
99 17 0.6182577013969421
99 18 0.7310153841972351
99 19 0.6933552026748657
99 20 0.5860617756843567
99 21 0.5816972851753235
99 22 0.5598727464675903
99 23 0.5552341341972351
